<a href="https://colab.research.google.com/github/davidheram/Intro-to-NLP/blob/main/intro_to_nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("https://breathecode.herokuapp.com/asset/internal-link?id=932&path=url_spam.csv")

In [ ]:
df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [ ]:
df["is_spam"].value_counts()

,count
is_spam,
False,2303
True,696


In [ ]:
df =df.drop_duplicates()
df =df.reset_index(inplace=False, drop=True)

In [ ]:
df["is_spam"].value_counts()

,count
is_spam,
False,2125
True,244


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("WordNetLemmatizer")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading WordNetLemmatizer: Package
[nltk_data]     'WordNetLemmatizer' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess_url(url):
  url = url.lower()
  tokens = re.split(r'[\/\.\-\?\=\&\_]', url)


  tokens = [lemmatizer.lemmatize(t)
  for t in tokens
            if t and not t.isdigit()and t not in stop_words]
  return " ".join(tokens)

In [ ]:
df["clean_url"] = df["url"].apply(preprocess_url)

In [ ]:
from sklearn.model_selection import train_test_split
X= df["clean_url"]
y= df["is_spam"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state= 134)


In [ ]:
from numpy import vectorize
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(3,5), max_features=5000)

X_train_vec= vectorizer.fit_transform(X_train)
X_test_vec= vectorizer.transform(X_test)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

svm_model = LinearSVC()
svm_model.fit(X_train_vec, y_train)

LinearSVC()

In [ ]:
y_pred = svm_model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.94      0.96      0.95       425
        True       0.59      0.47      0.52        49

    accuracy                           0.91       474
   macro avg       0.76      0.72      0.74       474
weighted avg       0.90      0.91      0.91       474



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

def grid_svm(X_train, y_train):

    pipeline = Pipeline([
        ("vectorizer", CountVectorizer(analyzer="char")),
        ("svm", LinearSVC(random_state=42))])

    param_grid = {

        "vectorizer__ngram_range": [(3,5), (4,6), (5,7)],
        "vectorizer__max_features": [2000, 3000, 5000],
        "svm__C": [0.01, 0.1, 1],
        "svm__class_weight": [
            "balanced",
            {0: 0.3, 1: 0.7},
            {0: 0.2, 1: 0.8},
            {0: 0.1, 1: 0.9}]}

    cv = RepeatedStratifiedKFold(
        n_splits=3,
        n_repeats=1,
        random_state=141)

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=cv,
        scoring="f1",
        n_jobs=-1,
        error_score="raise")

    grid_search.fit(X_train, y_train)

    return grid_search.best_estimator_

In [ ]:
best_model = grid_svm(X_train,y_train)

In [ ]:
preds= best_model.predict(X_test)
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

       False       0.93      0.96      0.95       411
        True       0.69      0.54      0.61        63

    accuracy                           0.91       474
   macro avg       0.81      0.75      0.78       474
weighted avg       0.90      0.91      0.90       474

